<a href="https://colab.research.google.com/github/rajadevineni/Scrapping_Wiki_Travel/blob/master/WT_article_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Location Clasification**

In [5]:
#Rajasekhar Devineni

import pandas as pd
import sys
import requests
import bs4
import urllib
from bs4 import BeautifulSoup

# Importing PyGithub libraries to create files on Github from the script
import base64
#!pip install PyGithub
#from github import Github
from pprint import pprint

# **Importing places data from Github source respository to Pandas DataFrame**

***The Source file is obtained from https://simplemaps.com/data/world-cities with 15,000 cities on free plan.***

In [52]:
# Make sure you have the source file in the same directory as the python file.
cities_data = pd.read_csv('worldcities.csv')
cities_data

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6850,139.7514,Japan,JP,JPN,Tōkyō,primary,35676000.0,1392685764
1,New York,New York,40.6943,-73.9249,United States,US,USA,New York,NaN,19354922.0,1840034016
2,Mexico City,Mexico City,19.4424,-99.1310,Mexico,MX,MEX,Ciudad de México,primary,19028000.0,1484247881
3,Mumbai,Mumbai,19.0170,72.8570,India,IN,IND,Mahārāshtra,admin,18978000.0,1356226629
4,São Paulo,Sao Paulo,-23.5587,-46.6250,Brazil,BR,BRA,São Paulo,admin,18845000.0,1076532519
...,...,...,...,...,...,...,...,...,...,...,...
15488,Timmiarmiut,Timmiarmiut,62.5333,-42.2167,Greenland,GL,GRL,Kujalleq,NaN,10.0,1304206491
15489,Cheremoshna,Cheremoshna,51.3894,30.0989,Ukraine,UA,UKR,Kyyivs’ka Oblast’,NaN,0.0,1804043438
15490,Ambarchik,Ambarchik,69.6510,162.3336,Russia,RU,RUS,Sakha (Yakutiya),NaN,0.0,1643739159
15491,Nordvik,Nordvik,74.0165,111.5100,Russia,RU,RUS,Krasnoyarskiy Kray,NaN,0.0,1643587468


**Some cities have same names but in different countries or in different states, so we eliminate lesser important cities based on the population**

In [7]:
duplicates = set(cities_data[cities_data.duplicated(['city_ascii'],keep=False)]['city_ascii'])
print(len(duplicates))
#1100 cities have same name despite different locations

1100


**Sorting the cities on Alphabetical order and then on Decreasing population, then removing the all the cities with same names what has lower population**

In [8]:
# Sorting the cities Alphabetically and then on descending order of population 
cities_data = cities_data.sort_values(['city_ascii','population'], ascending=(True,False))
# keeping just the highest populated cities if they have same name
cities_data = cities_data.drop_duplicates(['city_ascii'], keep='first')

**Dropping all the unwanted columns**

In [9]:
cities_data = cities_data.drop(columns= ['city','lat','lng','iso2','iso3','admin_name','capital','population','id'])
cities_data

,city_ascii,country
2757,'s-Hertogenbosch,Netherlands
1316,A Coruna,Spain
2912,Aalborg,Denmark
10572,Aarau,Switzerland
1849,Aarhus,Denmark
...,...,...
5699,`Ajman,United Arab Emirates
5761,`Amran,Yemen
1958,`Ar`ar,Saudi Arabia
4163,`Ataq,Yemen


**Function "get_article_data", Collects Article data from Wiki Travel page**

In [10]:
def get_article_data(URL) -> str:
  try:
    res = requests.get(URL)
    res.raise_for_status()
    wiki = bs4.BeautifulSoup(res.text,"html.parser")
    #article_content = []
    article_content = ''
    #Looping through all the paragraph and concatinating them into a string.
    for i in wiki.select('p'):
      #article_content.append(i.getText())
      article_content += i.getText()
    return article_content
  except requests.exceptions.HTTPError as e:
    return 'NA'

# **Setting up the authentication to access public repositories.**

In [ ]:
username = "Your Username"
password = "Your Password"
# authenticate to github
g = Github(username, password)
# get the authenticated user
user = g.get_user()


# **Fetching article data of each city from Wiki**

In [11]:
# Create empty DataFrame
city_descriptions = pd.DataFrame(columns = ["City", "Country", "Description"])

In [44]:
# counter
counter = 0
for index, row in cities_data_2.iterrows():
  city = row['city_ascii'].replace(' ', '_')
  # creating a URL with city name
  URL = 'https://wikitravel.org/en/' + city
  # print(URL)
  # storing Article data into a string
  article_data = get_article_data(URL)
  # removing all the \n chars. values from the article data
  article_data = article_data.replace("\n", "")
  # length of article data
  len_article = len(article_data)
  if len_article >700:
    new_row = {'City':row['city_ascii'], 'Country': row['country'], 'Description':article_data}
    city_descriptions = city_descriptions.append(new_row, ignore_index=True)
    counter += 1
    print(counter, row['city_ascii'])

1 Luxor
2 Lynchburg
3 Lynden
4 Lynnwood
5 Lyon
6 Maasin
7 Maastricht
8 Mabaruma
9 Macapa
10 Macau
11 Maceio
12 Machala
13 Machilipatnam
14 Mackay
15 Madaba
16 Madeira
17 Madera
18 Madison
19 Madiun
20 Madona
21 Madrid
22 Madura
23 Mae Hong Son
24 Mae Sot
25 Maebashi
26 Magadan
27 Magdeburg
28 Magnitogorsk
29 Magway
30 Maha Sarakham
31 Mahajanga
32 Maidenhead
33 Maidstone
34 Maiduguri
35 Maine
36 Mainz
37 Maizuru
38 Majuro
39 Makassar
40 Makati City
41 Makawao
42 Makeni
43 Makhachkala
44 Malabo
45 Malaga
46 Malakal
47 Malang
48 Malargue
49 Malatya
50 Malaybalay
51 Male
52 Mali
53 Maliana
54 Malibu
55 Malindi
56 Malmo
57 Malta
58 Mamaroneck
59 Mammoth Lakes
60 Mamoudzou
61 Man
62 Manado
63 Managua
64 Manalapan
65 Manama
66 Manasquan
67 Manassas
68 Manaus
69 Manchester
70 Manchester-by-the-Sea
71 Mandalay
72 Mandaluyong City
73 Mandeville
74 Mangalore
75 Manhattan
76 Manhattan Beach
77 Manila
78 Manisa
79 Manistee
80 Manitou Springs
81 Manizales
82 Mannheim
83 Manta
84 Manteca
85 Manteo
8

632 Paramaribo
633 Paramus
634 Paranagua
635 Paraparaumu
636 Pardubice
637 Paris
638 Parkersburg
639 Parma
640 Parnaiba
641 Parnu
642 Paro
643 Parry Sound
644 Pasadena
645 Pasco
646 Pasig City
647 Passau
648 Pasto
649 Paterson
650 Pathankot
651 Pathein
652 Pathum Thani
653 Patiala
654 Patna
655 Patra
656 Pattani
657 Pavlodar
658 Pawtucket
659 Paysandu
660 Peace River
661 Pearl
662 Pecs
663 Pedernales
664 Peekskill
665 Peje
666 Pekalongan
667 Pekanbaru
668 Pella
669 Pelotas
670 Pemagatshel
671 Pematangsiantar
672 Pemba
673 Pembroke Pines
674 Pendleton
675 Penedo
676 Pennsville
677 Pensacola
678 Penticton
679 Penzance
680 Pereira
681 Permet
682 Perpignan
683 Perryton
684 Perryville
685 Perth
686 Peru
687 Perugia
688 Pescara
689 Peshawar
690 Petaluma
691 Peterborough
692 Petersburg
693 Petoskey
694 Petropavlovsk-Kamchatskiy
695 Petropolis
696 Petrozavodsk
697 Pevek
698 Phan Thiet
699 Phatthalung
700 Phayao
701 Phetchabun
702 Phetchaburi
703 Phichit
704 Philadelphia
705 Philipsburg
706 Phi

1192 Semarang
1193 Sen Monorom
1194 Sendai
1195 Seoul
1196 Sept-Iles
1197 Sequim
1198 Serang
1199 Seremban
1200 Serpukhov
1201 Serres
1202 Sesheke
1203 Setubal
1204 Severna Park
1205 Severomorsk
1206 Sevilla
1207 Seville
1208 Seward
1209 Sfax
1210 Shah Alam
1211 Shahrisabz
1212 Shaker Heights
1213 Shamokin
1214 Shanghai
1215 Shannon
1216 Shantou
1217 Shaoguan
1218 Shaoxing
1219 Sharjah
1220 Shawnee
1221 Sheboygan
1222 Sheffield
1223 Shenyang
1224 Shenzhen
1225 Shepherdstown
1226 Shepparton
1227 Sherbrooke
1228 Sheridan
1229 Shihezi
1230 Shijiazhuang
1231 Shillong
1232 Shimla
1233 Shimoga
1234 Shiprock
1235 Shiraz
1236 Shiyan
1237 Shizuoka
1238 Shkoder
1239 Shoreline
1240 Shorewood
1241 Short Hills
1242 Shreveport
1243 Shumen
1244 Shwebo
1245 Shymkent
1246 Si Racha
1247 Siauliai
1248 Sibenik
1249 Sibiu
1250 Sibu
1251 Sidi Bel Abbes
1252 Sidon
1253 Siem Reap
1254 Siena
1255 Sierra Vista
1256 Siesta Key
1257 Sigulda
1258 Sihanoukville
1259 Silchar
1260 Silver City
1261 Silver Spring
1262 

1775 Vinh
1776 Vinh Long
1777 Vipava
1778 Virginia
1779 Virginia Beach
1780 Visalia
1781 Visby
1782 Viseu
1783 Vista
1784 Vitoria
1785 Vitoria-Gasteiz
1786 Vitsyebsk
1787 Vittoriosa
1788 Vladikavkaz
1789 Vladimir
1790 Vladivostok
1791 Vlore
1792 Volgograd
1793 Vologda
1794 Volos
1795 Voronezh
1796 Vranje
1797 Vratsa
1798 Vrsac
1799 Vukovar
1800 Vung Tau
1801 Vyborg
1802 Wa
1803 Waco
1804 Wagga Wagga
1805 Waianae
1806 Wailea
1807 Wailuku
1808 Waingapu
1809 Waitakere
1810 Wakayama
1811 Walker
1812 Walla Walla
1813 Wallaroo
1814 Wallsend
1815 Walnut Creek
1816 Waltham
1817 Wanaka
1818 Wangaratta
1819 Wappingers Falls
1820 Warangal
1821 Ware
1822 Warrensburg
1823 Warrenton
1824 Warri
1825 Warrington
1826 Warrnambool
1827 Warsaw
1828 Wasco
1829 Wasilla
1830 Waterbury
1831 Waterford
1832 Watson Lake
1833 Waukesha
1834 Wausau
1835 Wawa
1836 Wayland
1837 We
1838 Weehawken
1839 Weifang
1840 Weihai
1841 Weinan
1842 Welcome
1843 Wellesley
1844 Wellington
1845 Wells
1846 Wembley
1847 Wenatchee
184

In [49]:
# shape of city description dataframe
city_descriptions.shape

(4016, 3)

In [46]:
# Creating a CSV file to store all city descriptions.
city_descriptions.to_csv(r'/Users/rajasekhardevineni/Documents/GitHub/Scrapping_Wiki_Travel/city_description.csv', index=False)

In [47]:
# creating HDFS file to store all city descriptions.
city_descriptions.to_hdf('city_description.h5', key='city', mode='a', data_columnslist = True)